# KineticsデータセットでECO用のDataLoaderを作成する

## 目標
1.	Kinetics動画データセットをダウンロードできるようになる
2.	動画データをフレームごとの画像データに変換できるようになる
3.	ECOで使用するためのDataLoaderを実装できるようになる

## Library

In [1]:
import os
from PIL import Image
import csv
import numpy as np

import torch
import torch.utils.data
from torch import nn

import torchvision

## 動画を画像データにしたフォルダへのファイルパスのリスト作成

9-4_2_convert_mp4_to_jpeg.ipynbで画像にした。

In [2]:
def make_datapath_list(root_path):
    '''
    動画を画像データにしたフォルダへのファイルパスリストを作成
    Params
    ---------
    root_path : str 
        データフォルダへのrootパス
        
    Returns
    ---------
    ret : video_list
        動画を画像にしたフォルダへのファイルパスリスト
    '''
    
    # 動画を画像データにしたフォルダへのファイルパスリスト
    video_list = []
    
     # root_pathにあるディレクトリ（クラスの種類）のパスを取得
    class_list = os.listdir(path=root_path)
    
    # 各クラスの動画フィアルを画像化したフォルダへのパスを取得
    for class_list_i in class_list:  # クラスごとのループ
        
        # クラスフォルダへのパスを取得
        class_path = os.path.join(root_path, class_list_i)
        
        # 各クラスのフォルダ内の画像ファイルを取得するループ
        for file_name in os.listdir(class_path):
            
            # ファイル名と拡張子に分割
            name, ext = os.path.splitext(file_name)
            
            # 拡張子が'.mp4'でないファイルは無視する
            if ext == '.mp4':
                continue
                
            # 動画ファイルを画像に分割して保存したフォルダのパスを取得
            video_img_directory_path = os.path.join(class_path, name)
            
            # video_listに追加
            video_list.append(video_img_directory_path)
            
    return video_list

# 動作確認
root_path = './data/kinetics_videos/'
video_list = make_datapath_list(root_path)
# 1番目と２番めの動画の画像が格納されているディレクトリ
print(video_list[0])
print(video_list[1])
            

## 動画の前処理クラスの作成

In [9]:
class VideoTransform():
    '''
    動画を画像にした画像ファイルの前処理クラス
    学習時と推論時で異なる動作をさせる →　DataAugmentation省略しているので同じではないか？
    動画を画像に分割しているため、分割された画像たちをまとめて前処理する点に注意
    '''
    
    def __init__(self, resize, crop_size, mean, std):
        self.data_transform = {
            'train': torchvision.transforms.Compose([
                # DataAugmentationは今回は省略
                GroupResize(int(resize)),              # 画像をまとめてresize
                GroupCenterCrop(crop_size),      # 画像をまとめてCenterCrop
                GroupToTensor(),                          
                GroupImgNormalize(mean, std),  # 標準化
                Stack()                                            # 複数画像をframes次元で結合させる
            ]),
            'val': torchvision.transforms.Compose([
                GroupResize(int(resize)),              # 画像をまとめてresize
                GroupCenterCrop(crop_size),      # 画像をまとめてCenterCrop
                GroupToTensor(),                          
                GroupImgNormalize(mean, std),  # 標準化
                Stack()                                            # 複数画像をframes次元で結合させる
            ])
        }
        
    def __call_(self, img_group, phase):
        '''
        Params
        ---------
        phase : 'train' or 'val'
            前処理のモードを指定
        '''
        
        return self.data_transform[phase](img_group)

In [14]:
# 前処理クラスで使用するクラスの定義

class GroupResize():
    '''
    画像をまとめてResizeする
    画像の短い方の辺の長さがresizeに変換される
    アスペクト比は保たれる
    '''
    
    def __init__(self, resize, interpolation=Image.BILINEAR):
        '''リスケールする処理を用意（torchvisionから）'''
        self.rescaler = torchvision.transforms.Resize(resize, interpolation)
        
    def __call__(self, img_group):
        '''リスケールをimg_group(リスト)内の各imgに実施'''
        return [self.rescaler(img) for img in img_group]
    
    
class GroupCenterCrop():
    '''
    画像をまとめてCenterCropする
    (crop_size, crop_size)の画像を切り出す
    '''
    def __init__(self, crop_size):
        '''center cropする処理を用意（torchvisionから）'''
        self.ccrop = torchvision.transforms.CenterCrop(crop_size)
        
    def __call__(self, img_group):
        '''center cropをimg_group（リスト）内の各imgに実施'''
        return [self.ccrop(img) for img in img_group]
    
class GroupToTensor():
    '''画像をまとめてテンソル化するクラス'''
    
    def __init__(self):
        '''テンソル化する処理を用意する'''
        self.to_tensor = torchvision.transforms.ToTensor()
        
    def __call__(self, img_group):
        '''テンソル化をimg_group（リスト）内の各imgに実施
        0-1ではなく、0-255で扱うため　 *255
        0-255で扱うのは学習済みデータの形式に合わせるため
        '''
        
        # 画像データ自体がテンソルになっていればそれをまとめているのがテンソルでなくリストでもいいのか？
        # いや、最後のStack（）で
        return [self.to_tensor(img)*255 for img in img_group]
    

class GroupImgNormalize():
    '''
   画像をまとめて標準化するクラス
   ''' 
    def __init__(self, mean, std):
        '''標準化する処理を用意（torchvisionから）'''
        self.normalize = torchvision.transforms.Normalize(mean, std)
    
    def __call__(self, img_group):
        '''標準化をimg_group（リスト）内の各imgに実施'''
        return [self.normalize(img) for img in img_group]
    

class Stack():
    '''画像を１つのテンソルにまとめるクラス'''
    
    def __call__(self, img_group):
        '''img_groupはtorch.Size([3, 224, 224])を要素とするリスト'''
        
        ret = torch.cat([(x.flip(dims=[0])).unsqueeze(dim=0) for x in img_group], dim=0)
        # x.flip(dims=[0])で色チャネルを変更　RGB →　BGR
        # unsqueeze(dim=0)で各画像にframes用の次元を追加
        # frames次元で結合
        
        return ret

## Datasetの作成

In [ ]:
# kinetics-400のラベル名をIDに変換する辞書と逆にIDをラベル名に変換する辞書を用意

def get_label_dictionary(label_dictionary_path='./video_download/kinetics_400_label_dictionary.csv'):
    label_id_dict = {}
    id_label_dict = {}
    
    with open(label_dictionary_path, encoding='utf-8_sig') as f:
        
        # 読み込み
        reader = csv.DictReader(f, delimiter=',', quotechar="")
        
        # 1行ずつ読み込み、辞書型変数に追加
        for row in reader:
            # ラベル名をIDに変換
            label_id_dict.setdefault(
                row['class_label'], int(row['label_id'])-1)
            # IDをラベルに変換
            id_label_dict.setdeafault(
                int(row['label_id'])-1, row['class_label'])
            
    return label_id_dict, id_label_dict

# 確認
label_dictionary_path = './video_download/kinetics_400_label_dictionary.csv'
label_id_dict, id_label_dict = get_label_dictionary(label_dictionary_path)
label_id_dict

In [17]:
class VideoDataset(torch.utils.data.Dataset):
    '''
    動画のDataset
    '''
    
    def __init__(self, video_list, label_id_dict, num_segments, phase, transform, img_tmpl='img_{:05d}.jpeg'):
        self.video_list = video_list                        # 動画画像のフォルダへのパスリスト
        self.label_id_dict = label_id_dict             # ラベル名をidに変換する辞書型変数 
        self.num_segments = num_segments    # 動画を何分割して使用するか決める
        self.phase = phase                                     # train or val
        self.transform = transform                       # 前処理クラス
        self.img_tmpl = img_tmpl                        # 読み込みたい画像のファイル名のテンプレート
        
    def __len__(self):
        '''動画数を返す'''
        return len(self.video_list)
    
    def __getitem__(self, index):
        '''前処理をした画像たちのデータとラベル、ラベルIDを取得'''
        imgs_transformed, label, label_id, dir_path = self.pull_item(index)
        return imgs_transformed, label, label_id, dir_path
    
    def pull_item(self, index):
        '''前処理した画像たちのデータとラベル、ラベルIDを取得'''
        
        # 1. 画像たちを読み込んでリストにする
        dir_path = self.video_list[index]           # 画像たちが格納されたディレクトリ
        indices = self._get_indices(dir_path)  # 読み込み画像idxを求める
        img_group = self._load_imgs(dir_path, self.img_tmpl, indices)  # リストに読み込み
        
        # 2. ラベルを取得しidに変換
        label = (dir_path.split('/')[3].split('/')[0])  # なぜ2回'/'でsplitするのか
        label_id = self.label_id_dict[label]  # idを取得
        
        # 3. 前処理の実施
        imgs_transformed = self.transform(img_group, phase=self.phase)
        
        return imgs_transformed, label, label_id, dir_path
    
    def _load_imgs(self, dir_path, img_tmpl, indices):
        '''画像をまとめて読み込み、リスト化する'''
        
        img_group = []  # 画像格納用
        
        for idx in indices:
            # 画像パスを取得
            file_path = os.path.join(dir_path, img_tmpl.format(idx))
            
            # 画像読み込み
            img = Image.open(file_path).convert('RGB')
            
            # リストに追加
            img_group.append(img)
            
        return img_group
    
    def _get_indices(self, dir_path):
        '''動画全体をself.num_segmetに分割した際に取得する動画のindexのリストを取得'''
        
        # 動画のフレーム数を求める
        file_list = os.listdir(path=dir_path)  # ディレクトリ全部読み込んで
        num_frames = len(file_list)  # 何個画像があるか
        
        # 動画の取得間隔幅を求める
        tick = (num_frames) / float(self.num_segments)
        
        # 動画の取得間隔幅で取り出す際のindexをリストで求める
        indices = np.array([int(tick/2.0 + tick*x) for x in range(self.num_segments)]) + 1
        # 250frameで16frame抽出の場合
        # indices = [8 24 40 55 71 86 102 118 133 149 165 180 196 211 227 243]
        
        return indices
        

In [18]:
a = 'aasdf/fgh/hjk/34556/asdf'
a.split('/')

['aasdf', 'fgh', 'hjk', '34556', 'asdf']

In [ ]:
# 動作確認
# video_listの作成
root_path = './data/kinetics_videos/'
video_list = make_datapath_list(root_path)

# 前処理の設定
resize, crop_size = 224, 224
mean, std = [104, 117, 123], [1,1,1]
video_transform = VideoTransform(resize, crop_size, mean, std)

# Datasetの作成
# num_segments : 動画を何分かるして使用するか決める
val_dataset = VideoDataset(video_list, label_id_dict, num_segments=16, phase='val', transform=video_transform, img_tmpl='image_{:05d}.jpg')

# データの取り出し例
index = 0
print(val_dataset.__getitem__(index)[0].shape)  # 画像たちのテンソル
print(val_dataset.__getitem__(index)[1])             # ラベル名 
print(val_dataset.__getitem__(index)[2])             # ラベルID
print(val_dataset.__getitem__(index)[3])             # 動画へのパス


## DataLoaderの作成

In [ ]:
# DataLoader
batch_size = 8
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 動作確認
batch_iterator = iter(val_dataloader)  # イテレータに変換
imgs_transformeds, labels, label_ids, dir_path = next(batch_iterator)  # 1番目のbatch
print(imgs_transformeds.shape)